# Lab 6: Working with global data

The goal of this lab is to practice extracting useful information from publicly available datasets.  Here we are considering global atmospheric data, but you might also have large datasets from ice sheets, oceans, or geological observations.  It is useful to have some more refined techniques to visualize, process, and summarize information from these large datasets.

Our science questions are *up to you*! You may choose from the following:
- What regions of South America have the highest average annual evaporation?
- Do oceans or land surfaces typically receive more precipitation?
- In what month is Northern Hemisphere snow cover the highest?


---
## Part 1. Demonstration & skills practice
This will draw on several processing methods we have used before, with a few new tweaks.

### 1. Download & load in data  

For the demo, download the global 2-meter air temperature dataset `ERA5_LowRes_Monthly_t2m.nc` from [this page](https://fabienmaussion.info/climate_system/download.html).  While you are there, download `ERA5_LowRes_Invariant.nc` -- we will use it later.

Place both files in an easily accessible folder.  The cell below assumes you have them in a folder called `data` within the repo holding this Jupyter notebook.

Import the packages we will use, and open the dataset.

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size

In [ ]:
ds = xr.open_dataset('./data/ERA5_LowRes_Monthly_t2m.nc')
ds

#### A map plot of average air temperature

Let's compute the time average of the air temperature and plot it on a map, as we have done before.

In [ ]:
t2_tavg = ds.t2m.mean(dim='time')
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();

---
### 2. Finer control on color maps

Smooth (continuous) color maps like the above "look nice", but the human eye is not trained to see such small differences in color. For example, it would be quite difficult to tell the temperature of the Peruvian coast (above 280K? or below?). Sometimes, **discrete levels** are the way to go:  

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), levels=[240, 260, 280, 285, 290, 295, 300]) 
ax.coastlines(); ax.gridlines(); 

We can define either the specific levels, as above, or a number of levels.  Let's explore how many levels are needed to display the data most effectively.

In [ ]:
# live code here

### Selecting color map settings

Let's make a new variable called ``t2c_tavg``, which is ``t2_tavg`` converted to degrees celsius:

In [ ]:
t2c_tavg = t2_tavg - 273.15

Copy the plotting code from above and use it to plot `t2c_tavg` instead of `t2_tavg`.

In [ ]:
## live code here

What happened to the plot? Note the location of the 0 on the colorbar and the automated choice of a new colorscale. Note also that the data range is mostly dictated by very cold temperatures in Antarctica. The automated choices are a good first indication, but not always the most meaningful representation of the data. 

Let's see what happens with the below:

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
t2c_tavg.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdBu_r', center=False, 
              vmin=-40, vmax=20, levels=7, cbar_kwargs={'label': '°C'}) 
ax.set_title('Average annual 2m air temperature, ERA5 1979-2018')
ax.coastlines(); ax.gridlines(); 

A list of matplotlib's color tables can be found [here](http://www.matplotlib.org/examples/color/colormaps_reference.html). 

---
## 3. Extracting time series

Remember the selection methods we learned in Lab 2?  We can also use them for time series.  Let's first check the time period and sampling frequency of our data.

In [ ]:
# Lizz to write live code here

Let's define the variable `t2` for easy access to the temperature, and make a selection of a specific year.

In [ ]:
t2 = ds.t2m
t2.sel(time='2008')

The selection methods are very flexible.  **Make a prediction: what do you think each of the following commands will do?  Test them out and see.**
- t2.sel(time='2008-02')
- t2.sel(time='2008/02')
- t2.sel(time=slice('2008', '2012'))

In [ ]:
# Your answer here

### 3.1. Time series of globally averaged fields 

Do you remember the average air temperature on Earth?  I know you do.  Let's extract a global average from this ERA5 data and compare it with our prior knowledge.

Recall that to compute a global average on a sphere, we first need to weight the data by latitude (go back to [Lab 2](https://github.com/ehultee/climdyn-labs/blob/main/02-FM-NetCDF_Data.ipynb) if you need a refresher).

In [ ]:
## Define the meridional weights
weight = np.cos(np.deg2rad(ds.latitude))
weight = weight / weight.sum()

## Compute the zonal mean first
zonal_mean_t2_c = t2.mean(dim='longitude') - 273.15  # convert into Celsius

## Apply meridional weighting
weighted_zonal_mean_t2_c = zonal_mean_t2_c * weight
weighted_ts_t2_c = weighted_zonal_mean_t2_c.sum(dim='latitude')


Notice the second to last line in the cell above.  Although `zonal_mean_t2_c` is of dimensions (`time`, `latitude`), we successfully multiplied it by an array of dimension `latitude` only. This is called "broadcasting" in the numpy jargon. In general we need to be careful to match dimensions when we apply mathematical operations, but Xarray knows about the dimensions of your data (and their coordinates) and will always make arithmetic operations match these -- another good reason to use this package.

Let's examine the average surface temperature over the time period of the data.

In [ ]:
weighted_ts_t2_c.plot();

Logically, the global average temperature on Earth would be: 

In [ ]:
weighted_ts_t2_c.mean(dim='time')

**Q: Does the global average reflect what we discussed in previous weeks?**  
Questions like these are called "sanity checks" and help us confirm our code is producing sensible answers.

*...your answer here*

### 3.2. Resampling time series data

Resampling is the operation of changing the *sampling* of the data, i.e. the frequency at which it is sampled. One of the most meaningful ways to resample is to do an average, for example over each year:

In [ ]:
tsa_t2_c = weighted_ts_t2_c.resample(time='AS').mean()

In [ ]:
tsa_t2_c.plot()

Note that averaging is [not the only way](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html) available to resample data. **Try ``.std()`` and  ``.max()``, too.**

### 3.3. Computing a monthly climatography (or annual cycle)

Another way to look at time series data is to average them according to the time of year to study the annual cycle. This is done with the ``.groupby()`` method:

In [ ]:
tsm_t2_c = weighted_ts_t2_c.groupby('time.month').mean()

Let's check that `tsm_t2_c` is what we think it is. What is the new dimension of the data? What does it look like when we plot it?

In [ ]:
# Lizz to write live code here
tsm_t2_c.plot()

### 3.4. Averages and anomalies 

Remember when you found the temperature anomaly of Middlebury observations compared with a long-term average?  We can decompose any time series $A(t)$ into a reference value (often the mean) $\overline{A}$ and an anomaly $A'(t)$:

$$A(t) = \overline{A} + A'(t)$$

Often the variable t is omitted from the notation, i.e. one writes $A = \overline{A} + A'$.

#### *Exercise:* 
Compute and plot the global mean temperature anomaly for the year 1997 with respect to the 1979-2018 average.
This is a multi-step task; you and your partner may want to write out a plan on paper first.

In [ ]:
# your answer here

---
## 4. Selecting specific areas of our data

In the section above, we selected snapshots in time to generate time series.  We can also select spatial slices out of the data:

In [ ]:
t2_reg = t2_tavg.sel(longitude=slice(-40, 40))

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree()) # Note that I changed the projection
t2_reg.plot(ax=ax, transform=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.BORDERS); # What do you think this command does? 
ax.coastlines();

#### Live coding together 
Create a new variable, `t2c_clipped`, which is a subset of `t2c_tavg` between the longitudes (-20, 60) and the latitudes (40, -40). Plot the result. (*note: yes, it is (40, -40) and not (-40, 40)*)

In [ ]:
# live code here
t2c_clipped = ...

### 4.1. Selection based on a condition: masking

What if we are interested into air temperature on land only, and want to remove the oceans from our analyses? For this we can use a technique called "masking".  We will load a file with the same spatial resolution as our data, but with the core purpose of identifying which areas are land and which are ocean. 

First, load in the "Invariant" data.

In [ ]:
nc_inv = xr.open_dataset('./data/ERA5_LowRes_Invariant.nc')
nc_inv

In the `lsm` ("land-sea mask") variable, "1" means land, "0" means ocean. We are going to use this information to exclude ("mask out") the values of our dataset over the ocean:

In [ ]:
masked_t2_avg = t2c_tavg.where(nc_inv.lsm > 0.5)

In [ ]:
## inspect on a map plot
ax = plt.axes(projection=ccrs.Robinson())
masked_t2_avg.plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();

#### *Exercise*:
Compute the **zonal average** of land surface temperature and plot it.  Your plot should have axes of temperature versus latitude.  Compare your plot with a zonal average of global (land and ocean) surface temperature.

---
## Part 2: Exploring a new dataset

### Lab Procedure
Choose one of the science questions above.  Choose the appropriate dataset to address your question from the pre-processed "Monthly surface (3D)" ERA5 datasets linked on [this page](https://fabienmaussion.info/climate_system/download.html) and download it.  With your partner, make a plan to explore this dataset.  You might want to draft your plan on paper before starting to code.  

At minimum, the code and text you write below should allow you to:
1. Search the internet for basic information about the data source.  Answer the following questions:
    - What is ERA5? 
    - What institution provides the data? 
    - What time period is it available for?  
2. Load in the dataset; identify what variable(s) are included and their physical meaning.
    - What are the dimensions of the variables?
    - What is the spatial resolution of this dataset?
    - What is its temporal resolution?
    - What is the data type of the various variables and coordinates available in the dataset? What are the differences between each data type?
3. Perform a "sanity check" to confirm the data have loaded in correctly.
    - Consider making a plot, computing average/max/min values, time slices, etc.
4. Use an appropriate technique (global plot, regional plot, monthly climatology, selecting or averaging over certain dimensions) to address your science question.
5. Write text directly responding to the science question based on your analysis.
6. Time allowing: propose another science question that could be investigated with your dataset.

Remember to include **text cells describing the approach and interpretation**, and **comments in the code** to aid readability.

In [ ]:
## Add cells of code and markdown as needed here to complete the lab

---
## Endnotes
- ERA5 is a reanalysis product.  Learn more about reanalysis [here](https://www.ecmwf.int/en/research/climate-reanalysis).
- You may find the xarray documentation (...link...) useful
- This notebook is heavily based on Fabien Maussion's [Physics of the Climate System notebooks](https://fabienmaussion.info/climate_system/week_03/01_Lesson_MoreDataCrunching.html)
- Last update: 25 Mar 2024, Lizz Ultee

---
### Bonus method: quick data inspection with `imshow`

xarray's `.plot` method internally uses matplotlib's [pcolormesh](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html) which, for reasons too long to explain here, is the more accurate way to represent gridded data on a map. If you have a big dataset and you are willing to sacrifice some accuracy to see it faster, you can also use [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html):

In [ ]:
t2_tavg = ds.t2m.mean(dim='time')
ax = plt.axes(projection=ccrs.Robinson())
t2_tavg.plot.imshow(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines();

This plot should render about 4 times faster than the default plot, which is useful for data exploration. It should not be used for final rendering or for regional plots, though.